In [25]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [26]:
url = "https://www.hurun.net/zh-CN/Rank/HsRankDetails?pagetype=unicorn"
driver = webdriver.Chrome()
driver.get(url)

In [27]:
pagination = driver.find_element(By.CLASS_NAME, "pagination")
pages = pagination.find_elements(By.CLASS_NAME, "page-link")
for page in pages:
    print(page.text)

‹
1
2
3
4
5
...
68
›


In [28]:
pages = int(pages[-2].text)
pages

68

In [29]:
data = []
for page in range(1, pages+1):
    table = driver.find_element(By.ID, 'table')
    rows = table.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        row_data = [cell.text.strip() for cell in row.find_elements(By.TAG_NAME, 'td')]
        if row_data: data.append(row_data)
    
    next_page_button = driver.find_element(By.CLASS_NAME, 'pagination')
    next_page_button = next_page_button.find_elements(By.TAG_NAME, 'a')[-1]
    next_page_button.click()
    time.sleep(1)

len(data)

1360

In [38]:
df = pd.DataFrame(data, columns=['排名', '企业估值', '企业信息', '掌门人 / 联合创始人', '行业'])
df.head()

,排名,企业估值,企业信息,掌门人 / 联合创始人,行业
0,No.1,￥ 13800 亿,字节跳动,张一鸣,社交媒体
1,No.2,￥ 9450 亿,SpaceX,Elon Musk,航天
2,No.3,￥ 8300 亿,蚂蚁集团,井贤栋,金融科技
3,No.4,￥ 4500 亿,Shein,许仰天,电子商务
4,No.5,￥ 3800 亿,Stripe,John Collison、Patrick Collison,金融科技


In [39]:
df['排名'] = df['排名'].str.replace('No.', '').astype(int)
df['企业估值'] = df['企业估值'].str.replace(r'[^0-9]', '', regex=True).astype(int)
df.head()

,排名,企业估值,企业信息,掌门人 / 联合创始人,行业
0,1,13800,字节跳动,张一鸣,社交媒体
1,2,9450,SpaceX,Elon Musk,航天
2,3,8300,蚂蚁集团,井贤栋,金融科技
3,4,4500,Shein,许仰天,电子商务
4,5,3800,Stripe,John Collison、Patrick Collison,金融科技


In [33]:
df.to_excel('unicorns.xlsx', index=False)